# DeBERTa V2 Training with Focal Loss

**Purpose**: Fine-tune DeBERTa V2 XLarge using PyTorch for logical fallacy detection

**Dataset**: FLICC (Fallacy detection dataset with train/val/test splits)

**Method**: Full fine-tuning with Focal Loss (Gamma=4)

---

## Configuration Overview

Based on paper's findings:
- **Model**: microsoft/deberta-v2-xlarge
- **Learning Rate**: 1e-5 (paper-optimal)
- **Focal Loss Gamma**: 4.0 (critical for performance)
- **Weight Decay**: 0.01
- **Epochs**: 15
- **Batch Size**: 4 (with gradient accumulation of 4 = effective batch 16)
- **Device**: Apple Silicon MPS (Metal Performance Shaders)

## Step 1: Import Required Libraries

In [ ]:
# Core PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F

# HuggingFace Transformers
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    EarlyStoppingCallback
)

# Data handling
from datasets import Dataset, DatasetDict
import pandas as pd
import numpy as np

# Metrics
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    classification_report,
    confusion_matrix
)

# Utilities
import json
from pathlib import Path
from dataclasses import dataclass
from typing import Dict, List, Optional
import warnings
warnings.filterwarnings('ignore')

print("All libraries imported successfully!")

# Check device availability
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print(f"Using Apple Silicon GPU (MPS)")
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using CUDA GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("Using CPU")

print(f"PyTorch version: {torch.__version__}")

## Step 2: Training Configuration

### Paper-Validated Parameters

Based on research findings:
- **Focal Loss Gamma 4.0**: Critical for handling class imbalance
- **Learning Rate 1e-5**: Validated as optimal
- **15 Epochs**: Full convergence duration
- **Weight Decay 0.01**: Standard regularization

In [ ]:
@dataclass
class TrainingConfig:
    """
    Training configuration based on paper's best parameters.
    """
    
    # Model Configuration
    model_name: str = "microsoft/deberta-v2-xlarge"
    
    # Training Hyperparameters (Paper's Best)
    learning_rate: float = 1.0e-5        # Paper-validated optimal
    weight_decay: float = 0.01           # Standard regularization
    num_epochs: int = 15                 # Full convergence
    batch_size: int = 4                  # Small for stability
    gradient_accumulation_steps: int = 4 # Effective batch = 16
    
    # Focal Loss Configuration
    focal_gamma: float = 4.0             # Critical parameter (paper finding)
    
    # Data Paths
    train_data_path: str = "Data/fallacy_train.csv"
    val_data_path: str = "Data/fallacy_val.csv"
    test_data_path: str = "Data/fallacy_test.csv"
    
    # Output Configuration
    output_dir: str = "./output/deberta_flicc"
    
    # Training Options
    max_seq_length: int = 512            # Maximum sequence length
    warmup_ratio: float = 0.1            # 10% warmup
    seed: int = 42                       # Random seed
    logging_steps: int = 10              # Log every N steps
    save_strategy: str = "epoch"         # Save after each epoch
    evaluation_strategy: str = "epoch"   # Evaluate after each epoch
    
    # Early Stopping
    early_stopping_patience: int = 3     # Stop if no improvement for 3 epochs
    metric_for_best_model: str = "f1"    # Use F1 score for best model
    

# Initialize configuration
config = TrainingConfig()

# Display configuration
print("="*70)
print("TRAINING CONFIGURATION (Paper Parameters)")
print("="*70)
print(f"\nModel: {config.model_name}")

print(f"\nTraining Hyperparameters:")
print(f"  Learning Rate: {config.learning_rate} (paper-optimal)")
print(f"  Weight Decay: {config.weight_decay}")
print(f"  Epochs: {config.num_epochs}")
print(f"  Batch Size: {config.batch_size}")
print(f"  Gradient Accumulation: {config.gradient_accumulation_steps}")
print(f"  Effective Batch Size: {config.batch_size * config.gradient_accumulation_steps}")

print(f"\nFocal Loss Configuration:")
print(f"  Gamma: {config.focal_gamma} (critical for performance)")

print(f"\nData Splits:")
print(f"  Training: {config.train_data_path}")
print(f"  Validation: {config.val_data_path}")
print(f"  Test: {config.test_data_path}")

print(f"\nOutput Directory: {config.output_dir}")
print("="*70)

## Step 3: Load Data

Load all three data splits and prepare label mappings.

In [ ]:
def load_data(file_path: str, split_name: str) -> pd.DataFrame:
    """
    Load fallacy detection dataset.
    
    Args:
        file_path: Path to CSV file
        split_name: Name of split for display
        
    Returns:
        DataFrame with text and label columns
    """
    print(f"\nLoading {split_name} data from: {file_path}")
    df = pd.read_csv(file_path)
    print(f"Loaded {len(df):,} examples")
    
    # Display label distribution
    print(f"\nLabel distribution:")
    for label, count in df['label'].value_counts().sort_index().items():
        print(f"  {label:25s}: {count:4d} ({count/len(df)*100:.1f}%)")
    
    return df


# Load all splits
print("="*70)
print("LOADING DATA SPLITS")
print("="*70)

train_df = load_data(config.train_data_path, "TRAIN")
val_df = load_data(config.val_data_path, "VALIDATION")
test_df = load_data(config.test_data_path, "TEST")

# Create label mappings
unique_labels = sorted(train_df['label'].unique().tolist())
label2id = {label: idx for idx, label in enumerate(unique_labels)}
id2label = {idx: label for label, idx in label2id.items()}

print(f"\n" + "="*70)
print("LABEL MAPPINGS")
print("="*70)
print(f"\nNumber of classes: {len(unique_labels)}")
print(f"\nLabel to ID mapping:")
for label, idx in sorted(label2id.items()):
    print(f"  {idx}: {label}")
print("="*70)

## Step 4: Prepare Datasets for HuggingFace

Convert pandas DataFrames to HuggingFace Datasets.

In [ ]:
def prepare_dataset(df: pd.DataFrame) -> Dataset:
    """
    Convert DataFrame to HuggingFace Dataset.
    
    Args:
        df: DataFrame with text and label columns
        
    Returns:
        HuggingFace Dataset
    """
    # Add numeric labels
    df['labels'] = df['label'].map(label2id)
    
    # Create HuggingFace dataset
    dataset = Dataset.from_pandas(df[['text', 'labels']])
    
    return dataset


# Create datasets
print("\nCreating HuggingFace datasets...")

train_dataset = prepare_dataset(train_df)
val_dataset = prepare_dataset(val_df)
test_dataset = prepare_dataset(test_df)

# Create DatasetDict
dataset_dict = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})

print("\nDataset creation complete!")
print(dataset_dict)

## Step 5: Load Tokenizer and Tokenize Data

In [ ]:
print(f"\nLoading tokenizer: {config.model_name}")
tokenizer = AutoTokenizer.from_pretrained(config.model_name)
print("Tokenizer loaded!")


def tokenize_function(examples):
    """
    Tokenize text examples.
    
    Args:
        examples: Batch of examples with 'text' field
        
    Returns:
        Tokenized examples
    """
    return tokenizer(
        examples['text'],
        truncation=True,
        max_length=config.max_seq_length,
        padding=False  # Dynamic padding handled by data collator
    )


print("\nTokenizing datasets...")
tokenized_datasets = dataset_dict.map(
    tokenize_function,
    batched=True,
    desc="Tokenizing"
)

print("\nTokenization complete!")
print(tokenized_datasets)

## Step 6: Load Model

In [ ]:
print(f"\nLoading model: {config.model_name}")
print("This may take several minutes...\n")

model = AutoModelForSequenceClassification.from_pretrained(
    config.model_name,
    num_labels=len(unique_labels),
    id2label=id2label,
    label2id=label2id,
    problem_type="single_label_classification"
)

# Move model to device
model = model.to(device)

print("Model loaded successfully!")
print(f"\nModel parameters: {model.num_parameters():,}")
print(f"Device: {device}")

## Step 7: Define Focal Loss Trainer

Custom trainer implementing Focal Loss with Gamma=4.

**Focal Loss**: Addresses class imbalance by down-weighting easy examples.

Formula: `FL(pt) = -(1 - pt)^γ * log(pt)`

Where:
- `pt` is the probability of the correct class
- `γ` (gamma) controls the down-weighting factor

In [ ]:
class FocalLossTrainer(Trainer):
    """
    Custom Trainer with Focal Loss.
    
    Implements focal loss to handle class imbalance as per paper's findings.
    """
    
    def __init__(self, focal_gamma: float = 4.0, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.focal_gamma = focal_gamma
        print(f"\nUsing Focal Loss with Gamma = {self.focal_gamma}")
    
    def compute_loss(self, model, inputs, return_outputs=False):
        """
        Compute focal loss instead of standard cross-entropy.
        
        Args:
            model: The model being trained
            inputs: Input batch
            return_outputs: Whether to return model outputs
            
        Returns:
            Loss value (and outputs if requested)
        """
        labels = inputs.pop("labels")
        
        # Forward pass
        outputs = model(**inputs)
        logits = outputs.logits
        
        # Compute focal loss
        ce_loss = F.cross_entropy(logits, labels, reduction='none')
        pt = torch.exp(-ce_loss)  # Probability of true class
        focal_loss = ((1 - pt) ** self.focal_gamma * ce_loss).mean()
        
        return (focal_loss, outputs) if return_outputs else focal_loss


print("Focal Loss Trainer defined!")
print(f"Gamma parameter: {config.focal_gamma}")

## Step 8: Define Metrics

In [ ]:
def compute_metrics(eval_pred):
    """
    Compute evaluation metrics.
    
    Args:
        eval_pred: Tuple of (predictions, labels)
        
    Returns:
        Dictionary of metrics
    """
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    # Calculate metrics
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, 
        predictions, 
        average='weighted',
        zero_division=0
    )
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }


print("Metrics function defined!")

## Step 9: Setup Training Arguments

In [ ]:
# Create output directory
Path(config.output_dir).mkdir(parents=True, exist_ok=True)

# Define training arguments
training_args = TrainingArguments(
    # Output
    output_dir=config.output_dir,
    
    # Training hyperparameters
    learning_rate=config.learning_rate,
    weight_decay=config.weight_decay,
    num_train_epochs=config.num_epochs,
    per_device_train_batch_size=config.batch_size,
    per_device_eval_batch_size=config.batch_size,
    gradient_accumulation_steps=config.gradient_accumulation_steps,
    
    # Learning rate schedule
    warmup_ratio=config.warmup_ratio,
    lr_scheduler_type="linear",
    
    # Evaluation and saving
    evaluation_strategy=config.evaluation_strategy,
    save_strategy=config.save_strategy,
    load_best_model_at_end=True,
    metric_for_best_model=config.metric_for_best_model,
    greater_is_better=True,
    
    # Logging
    logging_dir=f"{config.output_dir}/logs",
    logging_steps=config.logging_steps,
    report_to=["tensorboard"],
    
    # Device configuration
    use_mps_device=(device.type == "mps"),
    
    # Reproducibility
    seed=config.seed,
    
    # Memory optimization
    fp16=False,  # MPS works better with FP32
    gradient_checkpointing=False,  # Can enable if OOM
    
    # Save settings
    save_total_limit=3,  # Keep only 3 best checkpoints
)

# Display training configuration
print("\n" + "="*70)
print("TRAINING ARGUMENTS")
print("="*70)
print(f"\nOutput directory: {training_args.output_dir}")
print(f"\nTraining:")
print(f"  Epochs: {training_args.num_train_epochs}")
print(f"  Batch size (per device): {training_args.per_device_train_batch_size}")
print(f"  Gradient accumulation: {training_args.gradient_accumulation_steps}")
print(f"  Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"  Learning rate: {training_args.learning_rate}")
print(f"  Weight decay: {training_args.weight_decay}")
print(f"  Warmup ratio: {training_args.warmup_ratio}")
print(f"\nEvaluation:")
print(f"  Strategy: {training_args.evaluation_strategy}")
print(f"  Metric for best model: {training_args.metric_for_best_model}")
print(f"\nDevice: {device}")
print("="*70)

## Step 10: Initialize Trainer and Start Training

**Expected Duration**: Full fine-tuning of DeBERTa V2 XLarge for 15 epochs will take many hours (potentially 10-20 hours on Apple Silicon).

The model will be evaluated after each epoch and the best model will be saved based on F1 score.

In [ ]:
# Data collator for dynamic padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Initialize trainer
trainer = FocalLossTrainer(
    focal_gamma=config.focal_gamma,
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=config.early_stopping_patience)]
)

print("\n" + "="*70)
print("STARTING TRAINING")
print("="*70)
print(f"\nModel: {config.model_name}")
print(f"Training samples: {len(tokenized_datasets['train']):,}")
print(f"Validation samples: {len(tokenized_datasets['validation']):,}")
print(f"\nThis will take many hours. Monitor the progress below.")
print(f"\nTensorBoard logs: {config.output_dir}/logs")
print(f"To view: tensorboard --logdir {config.output_dir}/logs")
print("\n" + "="*70 + "\n")

# Start training
train_result = trainer.train()

# Save the final model
print("\n" + "="*70)
print("TRAINING COMPLETE")
print("="*70)
print(f"\nSaving final model to: {config.output_dir}/final_model")
trainer.save_model(f"{config.output_dir}/final_model")
tokenizer.save_pretrained(f"{config.output_dir}/final_model")
print("Model saved!")

# Display training metrics
print("\nTraining Metrics:")
print(f"  Training Loss: {train_result.training_loss:.4f}")
print(f"  Training Steps: {train_result.global_step}")
print("="*70)

## Step 11: Validation Set Evaluation

In [ ]:
print("\n" + "="*70)
print("VALIDATION SET EVALUATION")
print("="*70)

# Evaluate on validation set
val_results = trainer.evaluate(eval_dataset=tokenized_datasets['validation'])

print("\nValidation Results:")
for key, value in val_results.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.4f}")
    else:
        print(f"  {key}: {value}")

# Get predictions for detailed analysis
val_predictions = trainer.predict(tokenized_datasets['validation'])
val_preds = np.argmax(val_predictions.predictions, axis=1)
val_labels = val_predictions.label_ids

# Print classification report
print("\nDetailed Classification Report (Validation):")
print(classification_report(
    val_labels,
    val_preds,
    target_names=[id2label[i] for i in range(len(id2label))],
    zero_division=0
))

print("="*70)

## Step 12: Test Set Evaluation (Final Performance)

In [ ]:
print("\n" + "="*70)
print("TEST SET EVALUATION (Final Performance)")
print("="*70)

# Evaluate on test set
test_results = trainer.evaluate(eval_dataset=tokenized_datasets['test'])

print("\nTest Results:")
for key, value in test_results.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.4f}")
    else:
        print(f"  {key}: {value}")

# Get predictions for detailed analysis
test_predictions = trainer.predict(tokenized_datasets['test'])
test_preds = np.argmax(test_predictions.predictions, axis=1)
test_labels = test_predictions.label_ids

# Print classification report
print("\nDetailed Classification Report (Test):")
print(classification_report(
    test_labels,
    test_preds,
    target_names=[id2label[i] for i in range(len(id2label))],
    zero_division=0
))

# Confusion matrix
print("\nConfusion Matrix (Test):")
cm = confusion_matrix(test_labels, test_preds)
print(cm)

print("="*70)

## Step 13: Save Training Report

In [ ]:
# Create comprehensive training report
training_report = {
    "model_configuration": {
        "model_name": config.model_name,
        "num_parameters": model.num_parameters(),
        "num_labels": len(unique_labels),
        "labels": unique_labels
    },
    "training_configuration": {
        "learning_rate": config.learning_rate,
        "weight_decay": config.weight_decay,
        "num_epochs": config.num_epochs,
        "batch_size": config.batch_size,
        "gradient_accumulation_steps": config.gradient_accumulation_steps,
        "effective_batch_size": config.batch_size * config.gradient_accumulation_steps,
        "focal_gamma": config.focal_gamma,
        "max_seq_length": config.max_seq_length,
        "warmup_ratio": config.warmup_ratio,
        "seed": config.seed
    },
    "data_splits": {
        "train_samples": len(train_df),
        "validation_samples": len(val_df),
        "test_samples": len(test_df)
    },
    "training_results": {
        "training_loss": float(train_result.training_loss),
        "total_steps": int(train_result.global_step)
    },
    "validation_results": {
        key: float(value) if isinstance(value, (int, float)) else value 
        for key, value in val_results.items()
    },
    "test_results": {
        key: float(value) if isinstance(value, (int, float)) else value 
        for key, value in test_results.items()
    },
    "label_mappings": {
        "label2id": label2id,
        "id2label": id2label
    }
}

# Save report
report_path = Path(config.output_dir) / "training_report.json"
with open(report_path, "w") as f:
    json.dump(training_report, f, indent=2)

print(f"\nTraining report saved to: {report_path}")

# Display summary
print("\n" + "="*70)
print("TRAINING SUMMARY")
print("="*70)
print(json.dumps(training_report, indent=2))
print("="*70)

## Training Complete!

### Summary

DeBERTa V2 XLarge has been successfully fine-tuned with Focal Loss (Gamma=4) for fallacy detection.

### Output Files

Located in `./output/deberta_flicc/`:
- `final_model/` - Fine-tuned model and tokenizer
- `training_report.json` - Complete training report
- `logs/` - TensorBoard logs
- Checkpoint directories for best models

### Performance Metrics

- **Validation Results**: See Step 11
- **Test Results**: See Step 12 (final unbiased performance)

### Usage Example

```python
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load fine-tuned model
model_path = "./output/deberta_flicc/final_model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Prepare text
text = "Your text here"
inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)

# Get prediction
with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
    predicted_class = torch.argmax(predictions, dim=-1).item()

print(f"Predicted fallacy: {id2label[predicted_class]}")
```

### Monitoring Training

To view training progress in TensorBoard:
```bash
tensorboard --logdir ./output/deberta_flicc/logs
```

### Next Steps

1. Analyze confusion matrix to identify challenging fallacy pairs
2. Review misclassified examples
3. Compare with LoRA-based Qwen model performance
4. Deploy the best performing model